# The 5 Puctured Sphere Term Rewriting System

In [1]:
%display latex
import string
from itertools import *
import re
import operator
import itertools
import functools
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
pool = ProcessPoolExecutor(8)
import concurrent.futures as futures
import pickle

## Defining the Free Algebra 

Let $s_A$ denote the simplest loop going around the punctures $A$.
The skein algebra is an algebra over $Q[q]$.
The variable $p$ is used in the `transform` function to allow more freedom in how coefficients act under symmetries.

In [2]:
G.<q> = FunctionField(QQ);
F.<p> = FunctionField(G)

It has central elements going around each puncture. For the 5-punctured sphere we therefore have the central elements $s_a, s_b, s_c, c_d$ and $s_{abcd}$, the loop around the final puncture. The variable $x,y,z,w$ are for finding errors in the relations.
The algebra $P$ is the centre of the skein algebra.

In [3]:
P.<s_a,s_b,s_c, s_d, s_abcd,x,y,z,w> = PolynomialRing(F,9,order='neglex')

We now define the free algebra $A$ which is generated by the extended generators: `s_bcd` is the simple loop around the punctures b,c,d; `s_bc_ad` consists of a loop around b and c and a loop around a and d; `s_b_c_d` is the loop which contains points b and d and goes around c on the outside. The skein algebra of the 5-punctured sphere is a quotient of this algebra. 

In [4]:
variables = ('s_ab, s_bc, s_cd, s_ad, '
             's_a_b_c, s_ab_cd, s_b_c_d, s_d_a_b, s_bc_ad, s_c_d_a, '
             's_a_bc_d, s_b_cd_a, s_c_da_b, s_d_ab_c, '
             's_ac, s_bd, s_abc, s_bcd, s_acd, s_abd').split(', ')
A = FreeAlgebra(P, order='neglex', names=variables)
for k, v in zip(variables, A._first_ngens(len(variables))):
    locals()[k] = v

In [5]:
s_ad_bc = s_bc_ad
s_cd_ab = s_ab_cd
s_c_ad_b = s_c_da_b

In [6]:
#Check ordering
s_ab < s_a_b_c < s_ac < s_abd

True

## Rewriting System

In this section we define a class `RewritingSystem` which consists of the term rewriting rules. This class has methods which are used to reduce an expression by using term rewriting rules.

The main functions are `replace` which makes a single reduction and `reduce_all` which reduces all the reduction rules: `reduce_all` is needed as the generated reduction rules will often have unreduced right hand sides.

The `to_gen_str` function takes an expression with powers and removes the powers e.g. `to_gen_str(s_ac^2) = s_ac*s_ac`. Firstly it replaces powers in the q coefficients: it places brackets around the result at this stage so that negative powers work correctly i.e. `to_gen_str(q^(-2)) = 1/(q*q)` rather than `1/q*q` which is just `1`. Then it removes powers in the monomials: at this stage it does not add brackets as brackets intefere with the regular expression used in `replace_rel`. 

In [7]:
_sub1 = re.compile(r"([q]+)\^([0-9]+)")
_sub2 = re.compile(r"([a-z_]+)\^([0-9]+)")

def to_gen_str(expr):
    """Turn an expression into a string with no powers."""
    s = str(expr)
    qsub = (_sub1.sub(
                 (lambda m: "("+("*".join([m.group(1)]*int(m.group(2))))+")"),
                 s))
    return _sub2.sub(
                 (lambda m: ("*".join([m.group(1)]*int(m.group(2))))),
                 qsub)

In [8]:
to_gen_str(s_a_b_c^2 * s_bcd)

's_a_b_c*s_a_b_c*s_bcd'

In [9]:
class RewritingSystem:
    def __init__(self, rels):
        self._rep_table = [
            (re.compile(f"{re.escape(to_gen_str(a))}(?![a-z_])"), f"({to_gen_str(b)})")
            for a, b in rels.items()
        ]
        self.rels = rels
        
    def __getitem__(self, key):
        """Return the reduction rule associated to the monomial key."""
        return self.rels[key]
    
    def __len__(self):
        """Count the number of reduction rules."""
        return len(self.rels)
    
    def __iter__(self):
        yield from self.rels
        
    def replace(self, expr):
        """Apply a single reduction to an expression."""
        expr_s1 = to_gen_str(expr)
        for pattern, res in self._rep_table:
            expr_s2 = pattern.sub(res, expr_s1, count=1)
            if expr_s1 != expr_s2:
                break
            expr_s1 = expr_s2
        return eval(expr_s2)
    
    def replace_with_rule(self, expr):
        """Apply a single reduction to an expression and return the result together with the reduction rule used."""
        expr_s1 = to_gen_str(expr)
        for (a, b), (pattern, res) in zip(self.rels.items(), self._rep_table):
            expr_s2 = pattern.sub(res, expr_s1, count=1)
            if expr_s1 != expr_s2:
                break
            expr_s1 = expr_s2
        return eval(expr_s2), {a: b}
    
    def reduce_all(self):
        """Reduces all the reduction rules using the RewritingSystem itself to do the reductions."""
        rw = {}
        reduced_bs = (pool.submit(self.reduce_rel, a, b) for a,b in self.rels.items())
        for fut in futures.as_completed(reduced_bs):
            a, b = fut.result()
            print(f"{a} : {b}")
            rw.update({a:b})
        return RewritingSystem(rw)
    
    def reduce_all_seq(self):
        """Reduces all the reduction rules like reduce_all but sequentially rather than inputing to the futures pool"""
        rw = {}
        reduced_bs = (self.reduce_rel(a, b) for a,b in self.rels.items())
        for (a,b) in reduced_bs:
            print(f"{a} : {b}")
            rw.update({a:b})
        return RewritingSystem(rw)
                      
    def reduce_rel(self, a, b):
        """Prints a and tries to reduce b using the RewritingSystem"""
        print(a)
        try:
            return a, reduce(b, self)
        except RecursionError:
            print(f"recursion error occurred on {a}")
            raise

## Reduction Functions

We now define the functions which reduce an expression using a RewritingSystem. The main function is `reduce(expr, rels, max_reductions)` but we also define `reduction_table(expr,rels, nrows)` which creates a table where each row applies a single reduction. To prevent infinite loops, for example when there is an error in the relations, these functions we stop after a set number of reductions.

In [10]:
class LoopException(Exception):
    pass

In [11]:
def multi_rep(expr, rels):
    """Create a generator object which applies a reduction at each interative step"""
    yield expr
    expr1 = rels.replace(expr)
    if expr == expr1:
        return
    expr = expr1
    yield from multi_rep(expr, rels)

In [12]:
def reduce(expr, rels, max_reductions=None):
    """Fully reduces an expression using a RewritingSystem with a max_reductions cap"""
    for i, res in enumerate(multi_rep(expr, rels)):
        if max_reductions is not None and i >= max_reductions:
            raise LoopException()
    return res

In [13]:
def multi_rep_with_rule(expr, rels, rule=None):
    """Similar to multi_rep but also outputs the rule applied"""
    yield expr, rule
    expr1, rule = rels.replace_with_rule(expr)
    if expr == expr1:
        return
    expr = expr1
    yield from multi_rep_with_rule(expr, rels, rule=rule)

In [14]:
def reduction_table(expr,rels, nrows):
    """Reduce an expression and create a table of each step."""
    reductions = itertools.islice(
    multi_rep_with_rule(expr, rels), 0, nrows+1)
    return table([["","Reduction table for "+str(expr),"reduction rule"]]+[[i, red, rule] for i, (red, rule) in enumerate(reductions)], header_row=True)

In [15]:
reduction_table(s_a*s_b*s_a*s_b^2, RewritingSystem({s_a*s_b: s_c}), 5)

,Reduction table for s_a^2*s_b^3,reduction rule
,,
,,
,,


Instead of stopping after a set number of interations one could also time out the function after a set amount of time. We do not use this function at the moment.

In [16]:
thread_pool = ThreadPoolExecutor(100)

def run_or_timeout(f, timeout=None, timeout_value=None):
    """Runs a function f for a set maximum amount of time"""
    try:
        return thread_pool.submit(f).result(timeout=timeout)
    except:
        return timeout_value

## Ambiguity Checking Functions

In this section we define the functions which are used to check overlap ambiguities i.e. expressions $abc$ where both $ab$ and $bc$ have a replacement rule. The main function is `check_amb(a, b, c, rels, max_reductions)` which checks this ambiguity.

In [17]:
def left_reduce(a,b,c,rels, max_reductions=None):
    """Reduce abc starting with the left ab reduction"""
    if a*b in rels:
        return reduce(RewritingSystem({a*b: rels[a*b]}).replace(a*b*c),rels, max_reductions=max_reductions)
    else: 
        return "no relation"
    
def right_reduce(a,b,c,rels, max_reductions=None):
    """Reduce abc starting with the right bc reduction"""
    if b*c in rels:
        return reduce(RewritingSystem({b*c: rels[b*c]}).replace(a*b*c),rels)
    else: 
        return "no relation"

The `amb_reduction_table` is similar to `reduction_table` but only works for the expression $abc$ and takes an extra argument which is either `"l"` or `"r"` and produces the reduction table for either the left or the right reduction of $abc$.

In [18]:
def amb_reduction_table(a,b,c,lr, rels, ncols):
    """Produce a left or right reduction table for the expression abc"""
    if lr == "l":
        leftreductions = itertools.islice(
            multi_rep_with_rule(RewritingSystem({a*b: rels[a*b]}).replace(a*b*c), rels), 0, ncols+1)
        return table([["","Reduction table for "+"("+str(a*b)+")*"+str(c), "left reduction rule"]]+[[i, red, rule] for i, (red, rule) in enumerate(leftreductions)], header_row=True)
    elif lr == "r":
        rightreductions = itertools.islice(
            multi_rep_with_rule(RewritingSystem({b*c: rels[b*c]}).replace(a*b*c), rels), 0, ncols+1)
        return table([["","Reduction table for "+str(a)+"*("+str(b*c)+")", "right reduction rule"]]+[[i, red, rule] for i, (red, rule) in enumerate(rightreductions)], header_row=True)
    else:
        print("must specify left l or right r")

The function `check_amb` checks the ambiguity $abc$. If the ambiguity is resolvable is returns `"consistent"`. If there is no relation a*b or b*c then it returns `"no relation"`. It raises a `LoopException` if the number of reductios exceeds a set amount.

In [19]:
def check_amb(a, b, c, rels, max_reductions=20):
    """Checks that abc can be unambiguously reduced."""
    if (a*b in rels) and (b*c in rels):
        try:
            l = left_reduce(a,b,c,rels, max_reductions)
            r = right_reduce(a,b,c,rels, max_reductions)
            if l == r:
                return "consistent"
            else:
                diff = l - r
                return diff
        except LoopException:
            return "max reductions exceeded"
    else: 
        return "no relation"

## Auxilary Functions

In [20]:
def s(lst):
    """Construct a loop from a list specifying the points"""
    return eval('s'+"".join(['_'+"".join(sorted(p)) for p in lst]))

def letters(a):
    """Return all the point letters in the string a."""
    return set(re.findall(r'[abcd]', str(a)))

def loop_points(loop):
    """Return all points which are used to define the loop."""
    return re.split(r'\_', str(loop))[1:]

def loop_point_groups(m):
    """Extract the subscript of a loop."""
    return re.findall(r'[abcd]+', str(m))

In [21]:
[s(["a", "bc", "d"]), s(["ad"]), s(["ab", "cd"])]

[s_a_bc_d, s_ad, s_ab_cd]

In [22]:
loop_points(s_a_b_c)

['a', 'b', 'c']

In [23]:
loop_point_groups(s_c_ad_b)

['c', 'da', 'b']

In [24]:
def make_subject(lhs, rhs, term):
    """Make the monomial term the subject of the equation lhs=rhs."""
    if term in lhs.monomials():
        return make_subject(rhs, lhs, term)
    if term in rhs.monomials():
        coeff = [c for (t,c) in list(rhs) if t == term]
        return {term: 1/coeff[0]*(lhs - rhs + coeff[0]*term)}
    return "monomial not in expression"

In [25]:
def terms_of_product(monomial):
    """Return a list of the loops in a monomial."""
    return eval('[' + str(monomial).replace('*', ',', -1).replace('^', '**', -1) + ']')

In [26]:
terms_of_product(s_ad*s_a_b_c*s_ab_cd)

[s_ad, s_a_b_c, s_ab_cd]

In [27]:
def is_ordered(mon):
    """Test if a monomial is ordered."""
    return terms_of_product(to_gen_str(mon)) == sorted(terms_of_product(to_gen_str(mon)))

def ordered_key(key):
    """Order the monomial key."""
    return sorted(terms_of_product(key)) == terms_of_product(key)

def reverse(k):
    """Reverse the terms in a monomial."""
    lst = terms_of_product(to_gen_str(k))
    lst.reverse()
    return prod(lst)

In [28]:
is_ordered(s_ad*s_ab^2) == False

True

In [29]:
is_ordered(s_bc*s_cd) == True

True

In [30]:
ordered_key(s_ab*s_cd) == True

True

In [31]:
ordered_key(s_cd*s_ab) == False

True

Lists of simple generators and non-simple generators:

In [32]:
simple_terms = [loop for loop in A.gens() if len(loop_point_groups(loop))==1]

In [33]:
m_terms = [loop for loop in A.gens() if not loop in simple_terms]

## Symmetry Functions

Give a relation we can generate other relations from it. Using `generate_symmetric` we can generate the relations formed by reflecting all the loops horizontally or vertically (and changing the coefficients) and using `generate_undercrossing` we generate the relation formed by changing all overcrossing to undercrossing and vice versa (and changing the coefficients).

In [34]:
def transform(expr, loop_fn=None, coeff_trans=1/q, p_trans=1/q):
    """Transform expr using loop_fn to transform the loops and coeff_trans and p_trans to change the p and q coeffients"""
    return sage_eval(str(expr), 
                     locals=dict(q=coeff_trans,p=p_trans,
                                 **{str(y): loop_fn(y) for y in P.gens()}, 
                                 **{str(z): loop_fn(z) for z in A.gens()},))

In [35]:
# Loop transformation functions
def swap(loop,pt1,pt2):
    """Transform a loop by swapping the points pt1 and pt2"""
    expr = str(loop)
    expr = re.sub(pt1, 'a1', expr)
    expr = re.sub(pt2, pt1, expr)
    return re.sub('a1', pt2, expr)

def horizontal_flip(loop):
    """Flip a loop horizontally. Assumes have 4 points."""
    lst = loop_points(swap(swap(loop, 'a', 'b'), 'c', 'd'))
    lst.reverse()
    return s(lst)

def vertical_flip(loop):
    """Flip a loop vertically. Assumes have 4 points."""
    lst = loop_points(swap(swap(loop, 'a', 'd'), 'b', 'c'))
    lst.reverse()
    return s(lst)

In [36]:
#tests
[horizontal_flip(s_a_b_c) == s_d_a_b,
vertical_flip(s_abc) == s_bcd,
 horizontal_flip(s_ab_cd) == s_ab_cd,
]

[True, True, True]

In [37]:
transform(q^2*s_c*s_abc + s_a + p, horizontal_flip)

(1/q+s_b) + 1/q^2*s_d*s_abd

In [38]:
def generate_symmetric(rel_dict):
    """Add to rel_dict all relations which are flips of the original relations."""
    gen_rels = {}
    for k,r in rel_dict.items():
        gen_rels.update({
        #both horizontal and vertical flip
        transform(transform(k, vertical_flip), horizontal_flip): transform(transform(r, vertical_flip), horizontal_flip),
        #horizontal flip
        transform(k, horizontal_flip): transform(r, horizontal_flip),
        #vertical flip
        transform(k, vertical_flip): transform(r, vertical_flip),
        #original relation
        k: r,
        })
    return gen_rels

In [39]:
def generate_undercrossing(rel_dict):
    """Add to rel_dict the undercrossing relations."""
    gen_rels = {}
    for k,r in rel_dict.items():
        m = terms_of_product(k)
        #Assume key of length 2 as that is all we need here.
        if len(m) == 2:
            gen_rels.update({m[0]*m[1]: transform(r, lambda x:x, q, q)})
            gen_rels.update({m[1]*m[0]: transform(r, lambda x:x, 1/q, q)})
    return gen_rels

## Term Rewriting System

We now define the term rewriting system which generated from the relations on the skein algebra of the four punctured sphere and where all the relations have keys of length 2.

#### Basic Relations 
These relations are the interesting relations for the skein algebra which are not generated from other relations.

In [40]:
simple = {
        #double cross relation 
        s_ac*s_bd: (q^(-2)*s_ab*s_cd + q^2*s_bc*s_ad
                 + q^(-1)*s_a*s_b*s_cd + q^(-1)*s_c*s_d*s_ab + q*s_a*s_d*s_bc + q*s_b*s_c*s_ad
                 + s_a*s_bcd + s_d*s_abc + s_c*s_abd + s_b*s_acd
                 + s_a*s_b*s_c*s_d + (q+q^(-1))*s_abcd
                ),
        #double and triple cross relation
        s_ac*s_bcd: (q*p*(s_bc*s_acd - p*s_abd - s_c*s_abcd - s_b*s_ad) 
                     + q^(-1)*p^(-1)*(s_cd*s_abc - p^(-1)*s_abd - s_c*s_abcd - s_d*s_ab) 
                     + s_c*s_abcd + s_a*s_b_c_d),
        
        #triple loop relations
        s_abc*s_acd: q*s_d_a_b + q^(-1)*s_b_c_d + s_abcd*s_ac + s_b*s_d,
        s_bcd*s_abc: q*s_a_bc_d + q^(-1)*s_ad + s_a*s_d + s_abcd*s_bc,
        s_abc*s_abd: q*s_d_ab_c + q^(-1)*s_cd + s_c*s_d + s_abcd*s_ab,
    
        #cubic
        s_a_b_c*s_ac: (  q^2*s_ab^2 + s_b^2 + q*s_a*s_b*s_ab + s_c^2
                       + s_abc^2 + q^(-2)*s_bc^2 + q^(-1)*s_a*s_bc*s_abc
                       + q*s_c*s_ab*s_abc + q^(-1)*s_b*s_c*s_bc + s_a*s_b*s_c*s_abc
                       - (q + q^(-1))^2 + s_a^2 ),
        #cubic with triple
        s_a_b_c*s_acd: ( q^2*s_ab*s_abd + s_b*s_bd + q*s_a*s_b*s_abd + s_c*s_cd
                        + s_abcd*s_abc + q^(-2)*s_bc*s_bcd + q^(-1)*s_a*s_abcd*s_bc
                        + q*s_c*s_abcd*s_ab + q^(-1)*s_b*s_c*s_bcd + s_a*s_b*s_c*s_abcd
                        + (q + q^(-1))*s_d + s_a*s_ad),
    
        #quadratic
        s_ab*s_b_cd_a: (q^2*s_acd^2 + s_cd^2 + q*s_a*s_cd*s_acd + s_b^2
                        + s_abcd^2 + q^(-2)*s_bcd^2 + q^(-1)*s_a*s_abcd*s_bcd
                        + q*s_b*s_abcd*s_acd + q^(-1)*s_b*s_cd*s_bcd + s_a*s_b*s_abcd*s_cd
                        - (q + q^(-1))^2 + s_a^2),
    
        s_bc*s_c_ad_b: (q^2*s_abd^2 + s_abcd^2 + q*s_c*s_abcd*s_abd + s_b^2
                        + s_ad^2 + q^(-2)*s_acd^2 + q^(-1)*s_c*s_ad*s_acd
                        + q*s_b*s_ad*s_abd + q^(-1)*s_b*s_abcd*s_acd + s_b*s_c*s_abcd*s_ad
                        - (q + q^(-1))^2 + s_c^2),
    
    }

simple_relations = generate_symmetric(generate_undercrossing(simple))

In [41]:
#length test
len(simple_relations) == 46

True

#### Commutative Relations
These are the relations of the form {ab: ba} which specify when two loops commute.

In [42]:
def generate_commutative():
    """Generate commutative relations."""
    commutative_pairs = [ 
        (s_ab, s_cd), (s_bc, s_ad), 
        (s_ab, s_abc), (s_bc, s_abc), (s_ac, s_abc), 
        #higher commutative
        (s_bc, s_a_bc_d), (s_ab, s_d_ab_c)
    ]
    rels = generate_symmetric({x[1]*x[0]: x[0]*x[1] for x in commutative_pairs})
    return {r[0]:r[1] for r in rels.items() if not is_ordered(r[0])}

In [43]:
commutative_rels = generate_commutative()

In [44]:
#length test
len(commutative_rels) == 18

True

In [45]:
#test all keys are not ordered
len(list(filter( lambda x: is_ordered(x) , commutative_rels.keys()))) == 0

True

In [46]:
def is_commutative(a, b):
    """Test if a and b commute."""
    return a*b in commutative_rels or b*a in commutative_rels or a==b or a in P or b in P

#### Commutators between Simple Loops

We now add all the relations of the form $[s_A, s_B]_q = (q^2 - q^{-2}) s_{A \cup B - A \cap B} + (q - q^{-1}) (s_{A \cap B} s_{A \cup B} + s_{A - A \cap B} s_{B - A \cap B})$ where $[s_A, s_B]_q := q s_A s_B - q^{-1} s_B s_A$ is the quantum Lie bracket.

In [47]:
def generate_commutator(a, b, r1, r2):
    """Given the monomials of the commutator generator the standard commutator."""
    if a>b:
        return (a*b, q^2*b*a + (q^(-1) - q^3)*r1 + (1-q^2)*r2)
    if b>a:
        return (b*a, q^(-2)*a*b + (q - q^(-3))*r1 + (1-q^(-2))*r2)
    raise ValueError(f"{a,b}")

In [48]:
def commutator_symmetries(lst):
    """Generate all the symmetric standard commutators given a list of standard commutator relations."""
    full = lst
    for a,b,r1,r2 in lst:
        full = full + list(filter(lambda y: {y[0], y[1]} != {a,b} ,[tuple(map(lambda x: transform(x, horizontal_flip), (b,a,r1,r2))),
                                               tuple(map(lambda x: transform(x, vertical_flip), (b,a,r1,r2))),
                                               tuple(map(lambda x: transform(transform(x, vertical_flip), horizontal_flip), (a,b,r1,r2)))]))
    return full

In [49]:
def generate_commutators():
    """Generates the standard commutators for the n=4 case."""
#    s_d_a_bc = p*(s_ad*s_abc -p*s_bcd - s_a*s_abcd - s_d*s_bc)
#    s_a_b_cd = p*(s_ab*s_bcd - p*s_acd - s_b*s_abcd - s_a*s_cd)
#    s_ab_c_d = p^(-1)*(s_cd*s_abc - p^(-1)*s_abd - s_c*s_abcd - s_d*s_ab)

    s_d_a_bc = q*(s_ad*s_abc -q*s_bcd - s_a*s_abcd - s_d*s_bc)
    s_a_b_cd = q*(s_ab*s_bcd - q*s_acd - s_b*s_abcd - s_a*s_cd)
    s_ab_c_d = q^(-1)*(s_cd*s_abc - q^(-1)*s_abd - s_c*s_abcd - s_d*s_ab)
    
    commutator_lst = [
    # doubles
    (s_ab, s_ad, s_bd, (s_a*s_abd + s_b*s_d)),
    (s_ab, s_ac, s_bc, (s_a*s_abc + s_b*s_c)),
    (s_bc, s_bd, s_cd, (s_b*s_bcd + s_c*s_d)),
    # double and triple
    (s_abc, s_ad, s_bcd, (s_a*s_abcd + s_d*s_bc)),
    (s_bcd, s_ab, s_acd, (s_b*s_abcd + s_a*s_cd)),
    #higher commutators
    (s_bc, s_d_ab_c, s_d_a_b, (s_b*s_d + s_c*s_d_a_bc)),
    (s_a_bc_d, s_a_b_c, s_cd, (s_c*s_d + s_a*s_a_b_cd)),
    (s_a_bc_d, s_ab, s_b_c_d, (s_b*s_d + s_a*s_ab_c_d)),
     ]
    rels = dict(generate_commutator(*x) for x in commutator_symmetries(commutator_lst))
    
    rels_system = RewritingSystem(rels)
    m_dict = RewritingSystem({s_ab*s_cd: s_ab_cd, s_bc*s_ad: s_bc_ad, s_cd*s_ab: s_ab_cd, s_ad*s_bc: s_bc_ad})
    backwards = RewritingSystem({ m_dict[k]:k  for k in m_dict})
    
    for k in [s_c_da_b*s_ab_cd, s_a_bc_d*s_ab_cd, s_b_cd_a*s_bc_ad, s_d_ab_c*s_bc_ad]:
        rels.update({k: reduce(reduce(reduce(k, backwards), rels_system), m_dict)})
    
    return rels


In [50]:
base_commutators = generate_commutators()

In [51]:
#length test
len(base_commutators) == 20

False

In [52]:
#test all keys are not ordered
len(list(filter( lambda x: is_ordered(x) , base_commutators.keys()))) == 0

True

#### Generate Extra Generators

These are the relations which generate the extra generators from the orginal simple generators.

In [53]:
def generate_m_generators():
    """Generate the relations used to generate the extended generators from the basic ones."""
    rels = { 
        # single missing point   
        s_ab*s_bc: q^(-1)*s_a_b_c + q*s_ac + s_a*s_c + s_b*s_abc,
        s_ab*s_cd: s_ab_cd,
        s_bc*s_ad: s_bc_ad,
        
        # two missing points
        s_ab*s_b_c_d: (q^(-1)*s_a_bc_d + s_a*s_d 
                   + q*p^(-1)*(s_cd*s_ac - p^(-1)*s_ad - s_a*s_d - s_c*s_acd)
                   + p^(-1)*s_b*(s_cd*s_abc - p^(-1)*s_abd - s_d*s_ab - s_c*s_abcd)),
        s_ad*s_a_b_c: (q^(-1)*s_d_ab_c + s_c*s_d 
                   + q*p^(-1)*(s_bc*s_bd - p^(-1)*s_cd - s_b*s_bcd - s_c*s_d)
                   + p^(-1)*s_a*(s_bc*s_abd - p^(-1)*s_acd - s_c*s_ad - s_b*s_abcd)), 
    }
    
    rels = generate_symmetric(generate_undercrossing(rels))
    
    rels2 = {
        # with commutative generators (all overcrossings changed to undercrossing)
        s_ab*s_bc_ad: q*s_ac*s_ad + q^(-1)*rels[s_a_b_c*s_ad] + s_b*s_abc*s_ad + s_a*s_c*s_ad,
        s_bc_ad*s_ab: q^(-1)*s_ad*s_ac + q*rels[s_ad*s_a_b_c] + s_b*s_ad*s_abc + s_a*s_c*s_ad,
        
        s_ab_cd*s_ad: q*s_ab*s_ac + q^(-1)*rels[s_ab*s_c_d_a] + s_d*s_ab*s_acd + s_a*s_c*s_ab,
        s_ad*s_ab_cd: q^(-1)*s_ac*s_ab + q*rels[s_c_d_a*s_ab] + s_d*s_acd*s_ab + s_a*s_c*s_ab,
    } 
    
    return {**rels, **generate_symmetric(rels2)}

In [54]:
m_relations = generate_m_generators()
m_rules = RewritingSystem(m_relations)

#### Rewriting Systems with Simple Relations

We construct the RewritingSystems used to generate the higher relations. 

`reordering_rw` consists of all the simple reordering relations $ba$ not including when there is a non-trivial relation between a and b e.g. a cross relation as they are in `simple_relations` and cannot be used to generate a higher commutator. 

In [55]:
reordering = {
    **commutative_rels,
    **generate_commutators(),
}

reordering_rw = RewritingSystem(reordering)

In [56]:
reordering[s_d_ab_c*s_bc]

(((q^2-1)/q^2)*s_b*s_d+((-q^2+1)/q)*s_a*s_c*s_abcd) + (((-q^2+1)/q)*s_c*s_d)*s_bc + ((q^4-1)/q^3)*s_d_a_b + ((-q^2+1)*s_c)*s_bcd + 1/q^2*s_bc*s_d_ab_c + (((q^2-1)/q)*s_c)*s_ad*s_abc

In [57]:
#Check only missing expected number of terms b*a
len([b*a for a, b in combinations(simple_terms, 2) if not b*a in reordering]) == 11

True

In [58]:
#test can now reorder all simple generators
len([b*a for a, b in combinations(simple_terms, 2) if not b*a in {**reordering, **simple_relations}]) == 0

True

We combine all the basic relations together into two RewritingSystems: `basic_m_rw` with the m_relations and `basic_rw` without.

In [59]:
basic_rels = {**reordering, **simple_relations}
basic_m_rels = {**reordering, **simple_relations, **m_relations}

In [60]:
basic_rw = RewritingSystem({**reordering, **simple_relations})
basic_m_rw = RewritingSystem(basic_m_rels)

#### Generate Higher Commutators

Autogenerate relations from the commutativity and commutator relations.

In [61]:
def s_term_prod(m):
    """Given a generator m generate the product of simple generators which generates m using the m_relations."""
    letters = re.findall(r'[abcd]', str(m))
    if m in [s_ab, s_bc, s_cd, s_ad, s_ac, s_bd, s_abc, s_bcd, s_acd, s_abd]:
        return m
    if m in [s_a_b_c, s_b_c_d, s_c_d_a, s_d_a_b]:
        sa = eval('s_'+"".join(sorted(letters[:2])))
        sb = eval('s_'+"".join(sorted(letters[1:])))
        return sa*sb
    if m in [s_ab_cd, s_bc_ad]:
        sa = eval('s_'+"".join(sorted(letters[:2])))
        sb = eval('s_'+"".join(sorted(letters[2:])))
        return sa*sb
    if m in [s_a_bc_d, s_b_cd_a, s_c_ad_b, s_d_ab_c]:
        sa = eval('s_'+"".join(sorted(letters[:2])))
        sb = eval('s_'+"".join(sorted(letters[1:3])))
        sc = eval('s_'+"".join(sorted(letters[2:])))
        return sa*sb*sc
    

def is_s_term_ordered(k):
    return is_ordered(prod([ s_term_prod(x) for x in terms_of_product(k) ]))

In [62]:
s_term_prod(s_c_ad_b)

s_cd*s_ad*s_ab

In [63]:
def generate_commutator(b, a):
    """Takes two generators a and b of which at least one is non-simple and generates a commutator relation
    based on the relations reordering_rw gives us on the simple generators."""
    if a>=b:
        raise ValueError(f'{b} is not larger than {a}')
    s1l = sorted(terms_of_product(s_term_prod(b))); s1 = prod(s1l)
    s2l = sorted(terms_of_product(s_term_prod(a))); s2 = prod(s2l)
    if is_ordered(s1*s2):
        print('case 1')
        print(s2*s1)
        l = reduce(s2, m_rules)*reduce(s1, m_rules)
        r = reduce(reduce(s2*s1, reordering_rw), m_rules)
        print(f'l is {l} and r is {r}')
        return make_subject(l, r, b*a)    
    if s1l[0] >= s2l[-1]:
        print(s1*s2)
        print('case 2')
        l = reduce(s1, m_rules)*reduce(s2, m_rules)
        r = reduce(reduce(s1*s2, reordering_rw), m_rules)
        print(f'l is {l} and r is {r}')
        return make_subject(l, r, b*a)
    return {}

In [64]:
generate_commutator(s_a_b_c, s_bc)

case 1
s_bc*s_ab*s_bc
l is s_a*s_c*s_bc + 1/q*s_bc*s_a_b_c + q*s_bc*s_ac + s_b*s_bc*s_abc and r is (((q^6-q^4-q^2+1)/q)*s_a*s_b) + ((q^8-2*q^4+1)/q^2)*s_ab + s_a*s_c*s_bc + (((q^6-q^4-q^2+1)/q)*s_c)*s_abc + (-q^5+q)*s_bc*s_ac + ((-q^2+1)*s_b)*s_bc*s_abc + q*s_a_b_c*s_bc + q^3*s_ac*s_bc + q^2*s_b*s_abc*s_bc


{s_a_b_c*s_bc: (((-q^6+q^4+q^2-1)/q^2)*s_a*s_b) + ((-q^8+2*q^4-1)/q^3)*s_ab + (((-q^6+q^4+q^2-1)/q^2)*s_c)*s_abc + 1/q^2*s_bc*s_a_b_c + q^4*s_bc*s_ac + q*s_b*s_bc*s_abc + (-q^2)*s_ac*s_bc + ((-q)*s_b)*s_abc*s_bc}

In [65]:
def higher_commutators():
    """Apply generate_commutator(a,b) to all relevant pairs."""
    gen_rels = {}
    for a, b in combinations(A.gens(),2):
        if not b*a in basic_rw and not b*a in m_rules:
            print(b,a)
            gen_rels.update(generate_commutator(b,a))
    return gen_rels

In [66]:
all_commutators = higher_commutators()

s_a_b_c s_ab
s_ab*s_bc*s_ab
case 2
l is s_a*s_c*s_ab + 1/q*s_a_b_c*s_ab + q*s_ac*s_ab + s_b*s_abc*s_ab and r is s_a*s_c*s_ab + q*s_ab*s_a_b_c + 1/q*s_ab*s_ac + s_b*s_ab*s_abc
s_ab_cd s_ab
s_ab*s_cd*s_ab
case 2
l is s_ab_cd*s_ab and r is s_ab*s_ab_cd
s_d_a_b s_ab
s_ab*s_ad*s_ab
case 2
l is s_b*s_d*s_ab + q*s_d_a_b*s_ab + 1/q*s_bd*s_ab + s_a*s_abd*s_ab and r is s_b*s_d*s_ab + 1/q*s_ab*s_d_a_b + q*s_ab*s_bd + s_a*s_ab*s_abd
s_a_b_c s_bc
case 1
s_bc*s_ab*s_bc
l is s_a*s_c*s_bc + 1/q*s_bc*s_a_b_c + q*s_bc*s_ac + s_b*s_bc*s_abc and r is (((q^6-q^4-q^2+1)/q)*s_a*s_b) + ((q^8-2*q^4+1)/q^2)*s_ab + s_a*s_c*s_bc + (((q^6-q^4-q^2+1)/q)*s_c)*s_abc + (-q^5+q)*s_bc*s_ac + ((-q^2+1)*s_b)*s_bc*s_abc + q*s_a_b_c*s_bc + q^3*s_ac*s_bc + q^2*s_b*s_abc*s_bc
s_b_c_d s_bc
s_bc*s_cd*s_bc
case 2
l is s_b*s_d*s_bc + 1/q*s_b_c_d*s_bc + q*s_bd*s_bc + s_c*s_bcd*s_bc and r is s_b*s_d*s_bc + q*s_bc*s_b_c_d + 1/q*s_bc*s_bd + s_c*s_bc*s_bcd
s_bc_ad s_bc
s_bc*s_ad*s_bc
case 2
l is s_bc_ad*s_bc and r is s_bc*s_bc_ad
s_ab

l is s_a*s_c*s_bd + 1/q*s_bd*s_a_b_c + q*s_bd*s_ac + s_b*s_bd*s_abc and r is (((-q^4+2*q^2-1)/q^2)*s_b^2*s_abcd) + ((q^2-1)*s_c*s_d)*s_ab + ((-q^2+1)*s_a*s_d)*s_bc + (((-q^4+2*q^2-1)/q^2)*s_b*s_c)*s_ad + ((q^4-1)/q)*s_ab_cd + ((-q^4+1)/q)*s_bc_ad + s_a*s_c*s_bd + (((-q^6+q^4+q^2-1)/q^3)*s_b)*s_acd + ((q^2-1)*s_b)*s_ab*s_bcd + (((-q^2+1)/q^2)*s_b)*s_bc*s_abd + 1/q*s_a_b_c*s_bd + q*s_ac*s_bd + s_b*s_abc*s_bd
s_abc s_a_b_c
s_abc*s_ab*s_bc
case 2
l is s_a*s_c*s_abc + 1/q*s_abc*s_a_b_c + q*s_abc*s_ac + s_b*s_abc^2 and r is s_a*s_c*s_abc + 1/q*s_a_b_c*s_abc + q*s_ac*s_abc + s_b*s_abc^2
s_bcd s_a_b_c
s_bcd*s_ab*s_bc
case 2
l is s_a*s_c*s_bcd + 1/q*s_bcd*s_a_b_c + q*s_bcd*s_ac + s_b*s_bcd*s_abc and r is (((q^6-q^4-q^2+1)/q)*s_c*s_abcd+(-q^2+1)*s_a*s_b*s_d) + ((-q^2+1)*s_b*s_abcd)*s_bc + (((q^6-q^4-q^2+1)/q)*s_b)*s_ad + ((-q^3+q)*s_a)*s_b_c_d + (((-q^2+1)/q)*s_a)*s_bd + s_a*s_c*s_bcd + ((q^8-2*q^4+1)/q^2)*s_abd + (-q^5+q)*s_bc*s_acd + q*s_a_b_c*s_bcd + q^3*s_ac*s_bcd + q^2*s_b*s_abc*s_bcd
s_abd

l is s_abd*s_bc_ad and r is (((q^2-1)/q^2)*s_b*s_abcd)*s_ad + (((q^2-1)/q^2)*s_c)*s_ad^2 + ((q^4-1)/q^3)*s_ad*s_acd + 1/q^2*s_bc_ad*s_abd
s_a_bc_d s_c_d_a
case 1
s_cd*s_ad*s_ab*s_bc*s_cd
l is (-s_a*s_b*s_c^2*s_abcd+((-q^2+1)/q)*s_a^2*s_c*s_d) + ((-q^2)*s_a*s_c)*s_ad + ((-1/q)*s_b*s_c*s_abcd+((-q^2+1)/q^2)*s_a*s_d)*s_c_d_a + 1/q^2*s_a*s_c*s_a_bc_d + ((-q)*s_b*s_c*s_abcd+(-q^2+1)*s_a*s_d)*s_ac + (-s_b*s_c*s_d*s_abcd+((-q^2+1)/q)*s_a*s_d^2+(-q)*s_a*s_c^2)*s_acd + ((-q)*s_a*s_b*s_c)*s_abd + q*s_a*s_c*s_ab*s_bd + s_a*s_c^2*s_ab*s_bcd + (-q)*s_c_d_a*s_ad + 1/q^3*s_c_d_a*s_a_bc_d + (-s_c)*s_c_d_a*s_acd + (-s_b)*s_c_d_a*s_abd + q*s_a*s_c*s_ac*s_cd + (-q^3)*s_ac*s_ad + 1/q*s_ac*s_a_bc_d + ((-q^2)*s_c)*s_ac*s_acd + ((-q^2)*s_b)*s_ac*s_abd + s_a*s_b*s_c*s_abc*s_cd + ((-q^2)*s_d)*s_acd*s_ad + 1/q^2*s_d*s_acd*s_a_bc_d + ((-q)*s_c*s_d)*s_acd^2 + ((-q)*s_b*s_d)*s_acd*s_abd + s_c_d_a*s_ab*s_bd + 1/q*s_c*s_c_d_a*s_ab*s_bcd + s_c_d_a*s_ac*s_cd + 1/q*s_b*s_c_d_a*s_abc*s_cd + q^2*s_ac*s_ab*s_bd + q*s_c*s_

l is (-s_b*s_c*s_abcd+((-q^2+1)/q)*s_a*s_d)*s_bd + (-q^2)*s_bd*s_ad + 1/q^2*s_bd*s_a_bc_d + ((-q)*s_c)*s_bd*s_acd + ((-q)*s_b)*s_bd*s_abd + q*s_bd*s_ab*s_bd + s_c*s_bd*s_ab*s_bcd + q*s_bd*s_ac*s_cd + s_b*s_bd*s_abc*s_cd and r is (((q^4-1)/q)*s_c*s_d*s_abcd+(-q^2+1)*s_a*s_b*s_d^2+(-q^2+1)*s_a*s_b*s_c^2) + ((q^4-1)/q)*s_ab + (((q^4-2*q^2+1)/q^2)*s_b*s_d*s_abcd+((-q^4+1)/q)*s_a*s_c)*s_bc + (((-q^4+2*q^2-1)/q^2)*s_b^2*s_abcd)*s_cd + (((q^4-3*q^2+2)/q)*s_b*s_c)*s_a_b_c + ((q^2-1)*s_c*s_d)*s_ab_cd + (((-q^2+1)/q)*s_a*s_d)*s_b_c_d + (((-q^4+2*q^2-1)/q)*s_b*s_c)*s_c_d_a + ((-q^4+1)/q)*s_b_cd_a + ((-q^3+q)*s_b*s_c)*s_ac + ((-q^2)*s_b*s_c*s_abcd+(-2*q^3+2*q)*s_a*s_d)*s_bd + (((q^4-1)/q^2)*s_c+((-q^2+1)/q^2)*s_b^2*s_c)*s_abc + ((-2*q^2+2)*s_a*s_c*s_d)*s_bcd + (((-q^4+2*q^2-1)/q^2)*s_b*s_c*s_d)*s_acd + ((q^4-1)*s_d+(-q^2+1)*s_b^2*s_d)*s_abd + ((-q^4+1)/q^2)*s_bc*s_ac + (((q^6-q^4-q^2+1)/q^3)*s_b)*s_bc*s_abc + (((-q^4+1)/q)*s_d)*s_bc*s_acd + (((-q^6+q^4+q^2-1)/q^3)*s_b)*s_cd*s_acd + (-2*q^4+1)*s_ad

l is ((-1/q^2)*s_c*s_d*s_abcd)*s_abc + (-1/q^2)*s_abc*s_ab + (((q^2-1)/q^2)*s_a*s_c)*s_abc*s_bc + 1/q^2*s_abc*s_b_cd_a + ((-1/q^3)*s_c)*s_abc^2 + ((-1/q)*s_d)*s_abc*s_abd + q*s_abc*s_bc*s_ac + s_d*s_abc*s_bc*s_acd + 1/q*s_abc*s_ad*s_bd + 1/q^2*s_c*s_abc*s_ad*s_bcd and r is ((q^5-2*q^3+q)*s_c*s_d^2+((q^6-3*q^4+q^2+1)/q^2)*s_a*s_b*s_d*s_abcd) + ((q^4-3*q^2+2)*s_b*s_d^2)*s_bc + ((q^6-q^4-2*q^2+2)*s_d)*s_cd + (((-q^2+1)/q^3)*s_a*s_abcd)*s_b_c_d + (((q^2-1)/q^2)*s_c*s_abcd)*s_bc_ad + ((q^2-1)*s_d)*s_d_ab_c + (((-q^2+1)/q)*s_a*s_abcd)*s_bd + ((-1/q^2)*s_c*s_d*s_abcd)*s_abc + (((q^8-2*q^6-q^4+q^2+1)/q^3)*s_b*s_d+((-q^2+1)/q^2)*s_a*s_c*s_abcd)*s_bcd + (((q^6-2*q^4+1)/q)*s_a*s_d)*s_acd + (-1/q^2)*s_ab*s_abc + (((-q^2+1)/q)*s_d)*s_bc*s_b_c_d + (((q^2-1)/q^2)*s_a*s_c)*s_bc*s_abc + (((-q^2+1)/q^2)*s_c*s_d)*s_bc*s_bcd + (((q^2-1)/q^2)*s_a*s_d)*s_bc*s_abd + (((-q^6+q^4+q^2-1)/q)*s_d)*s_ad*s_ac + ((-q^4+2*q^2-1)*s_b*s_d)*s_ad*s_abc + ((-q^4+1)/q^4)*s_b_c_d*s_bcd + ((q^4-1)/q^3)*s_bc_ad*s_abd + 1/q^2*

l is (-s_a*s_d*s_abcd)*s_abc + (-1)*s_abc*s_bc + s_abc*s_c_da_b + ((-q)*s_a)*s_abc^2 + ((-1/q)*s_d)*s_abc*s_bcd + q*s_abc*s_ab*s_ac + s_d*s_abc*s_ab*s_acd + 1/q*s_abc*s_cd*s_bd + s_a*s_abc*s_cd*s_abd and r is (((q^4-1)/q^2)*s_b*s_c*s_d*s_abcd+((q^4-2*q^2+1)/q)*s_a*s_d^2) + ((q^2-1)*s_b*s_d^2)*s_ab + ((q^4-q^2)*s_d)*s_ad + (((-q^2+1)/q^2)*s_a*s_abcd)*s_ab_cd + (((q^2-1)/q)*s_c*s_abcd)*s_d_a_b + (((-q^2+1)/q^2)*s_d)*s_a_bc_d + (((q^2-1)/q^3)*s_c*s_abcd)*s_bd + (-s_a*s_d*s_abcd)*s_abc + ((q^3-q)*s_c*s_d)*s_acd + (((q^6-1)/q^3)*s_b*s_d+((q^2-1)/q^2)*s_a*s_c*s_abcd)*s_abd + (((q^2-1)/q)*s_d)*s_ab*s_d_a_b + (((-q^2+1)/q^2)*s_c*s_d)*s_ab*s_bcd + (((q^2-1)/q^2)*s_a*s_d)*s_ab*s_abd + (-1)*s_bc*s_abc + ((-q^4+1)/q^3)*s_ab_cd*s_bcd + ((q^4-1)/q^2)*s_d_a_b*s_abd + s_c_da_b*s_abc + ((-q^3+q)*s_d)*s_ac*s_cd + ((q^4-1)/q^4)*s_bd*s_abd + ((-q^2+1)*s_b*s_d)*s_abc*s_cd + ((-q)*s_a)*s_abc^2 + ((-1/q)*s_d)*s_bcd*s_abc + (((q^4-1)/q^3)*s_a)*s_abd^2 + q*s_ab*s_ac*s_abc + s_d*s_ab*s_acd*s_abc + 1/q*s_cd*s_bd

l is (((q^2-1)/q^3)*s_c*s_d+(-1/q^2)*s_a*s_b*s_abcd)*s_bcd + (-1/q^4)*s_bcd*s_cd + (((q^2-1)/q^2)*s_a*s_c)*s_bcd*s_ad + s_bcd*s_d_ab_c + ((-1/q^3)*s_b)*s_bcd^2 + ((-1/q^3)*s_a)*s_bcd*s_acd + 1/q^3*s_bcd*s_bc*s_bd + 1/q^2*s_a*s_bcd*s_bc*s_abd + q*s_bcd*s_ac*s_ad + s_b*s_bcd*s_abc*s_ad and r is (((q^6-q^4+q^2-1)/q^2)*s_a*s_c*s_d*s_abcd+((q^8-2*q^6+2*q^2-1)/q^3)*s_a^2*s_b) + (((q^10-q^8-q^6+q^4+q^2-1)/q^4)*s_a)*s_ab + (((q^2-1)/q^2)*s_a^2*s_c)*s_bc + (((q^6-q^4-q^2+1)/q)*s_c*s_abcd+(q^4-2*q^2+1)*s_a*s_b*s_d)*s_ad + (((q^2-1)/q)*s_d*s_abcd)*s_a_b_c + ((-q^2+1)*s_b*s_abcd)*s_bc_ad + ((-q^2+1)*s_a)*s_b_cd_a + ((q^3-q)*s_d*s_abcd)*s_ac + ((q^2-1)*s_b*s_d*s_abcd+((q^8-q^6+q^4-1)/q^3)*s_a*s_c)*s_abc + (((q^2-1)/q^3)*s_c*s_d+(-1/q^2)*s_a*s_b*s_abcd)*s_bcd + (((q^8-q^6-q^4+2*q^2-1)/q^3)*s_a*s_d)*s_abd + ((-q^5+q^3)*s_a)*s_bc*s_ac + ((-q^4+q^2)*s_a*s_d)*s_bc*s_acd + (-1/q^4)*s_cd*s_bcd + (((q^6-q^4-q^2+1)/q)*s_b)*s_ad^2 + (((-q^2+1)/q^3)*s_a)*s_ad*s_bd + ((q^8-2*q^4+1)/q^2)*s_ad*s_abd + (((q^2-1)/

In [67]:
#List of missing reordering relations which are still to be generated.
[b*a for a,b in combinations(A.gens(),2) if not b*a in {**all_commutators, **basic_rels, **m_relations}]

[s_ab_cd*s_a_b_c,
 s_d_a_b*s_a_b_c,
 s_c_da_b*s_a_b_c,
 s_d_ab_c*s_a_b_c,
 s_b_c_d*s_ab_cd,
 s_d_a_b*s_ab_cd,
 s_bc_ad*s_ab_cd,
 s_b_cd_a*s_ab_cd,
 s_d_ab_c*s_ab_cd,
 s_d_a_b*s_b_c_d,
 s_bc_ad*s_b_c_d,
 s_b_cd_a*s_b_c_d,
 s_c_da_b*s_b_c_d,
 s_d_ab_c*s_b_c_d,
 s_bc_ad*s_d_a_b,
 s_c_d_a*s_d_a_b,
 s_a_bc_d*s_d_a_b,
 s_b_cd_a*s_d_a_b,
 s_d_ab_c*s_d_a_b,
 s_c_d_a*s_bc_ad,
 s_a_bc_d*s_bc_ad,
 s_c_da_b*s_bc_ad,
 s_b_cd_a*s_c_d_a,
 s_d_ab_c*s_c_d_a,
 s_b_cd_a*s_a_bc_d,
 s_c_da_b*s_a_bc_d,
 s_d_ab_c*s_a_bc_d,
 s_c_da_b*s_b_cd_a,
 s_d_ab_c*s_b_cd_a,
 s_d_ab_c*s_c_da_b]

In [68]:
reorder_basic = {**all_commutators, **basic_rels, **m_relations}
commutator_rw = RewritingSystem(reorder_basic)

In [69]:
%%time
pool = ProcessPoolExecutor(8)
reduced_commutator_rw = commutator_rw.reduce_all()

s_a_b_c*s_ab
s_d_a_b*s_ab
s_ab_cd*s_ab
s_a_b_c*s_bc
s_b_c_d*s_bc
s_bc_ad*s_bc
s_c_d_a*s_cd
s_a_b_c*s_ab : ((-q^2+1)*s_b*s_c) + ((-q^4+1)/q)*s_bc + ((-q^2+1)*s_a)*s_abc + q^2*s_ab*s_a_b_c
s_ab_cd*s_ab : s_ab*s_ab_cd
s_d_a_b*s_ab : (((q^2-1)/q^2)*s_a*s_d) + ((q^4-1)/q^3)*s_ad + (((q^2-1)/q^2)*s_b)*s_abd + 1/q^2*s_ab*s_d_a_b
s_a_b_c*s_bc : (((q^2-1)/q^2)*s_a*s_b) + ((q^4-1)/q^3)*s_ab + (((q^2-1)/q^2)*s_c)*s_abc + 1/q^2*s_bc*s_a_b_c
s_b_c_d*s_cd
s_ab_cd*s_cd
s_b_c_d*s_bc : ((-q^2+1)*s_c*s_d) + ((-q^4+1)/q)*s_cd + ((-q^2+1)*s_b)*s_bcd + q^2*s_bc*s_b_c_d
s_bc_ad*s_bc : s_bc*s_bc_ad
s_ab_cd*s_cd : s_cd*s_ab_cd
s_b_c_d*s_cd : (((q^2-1)/q^2)*s_b*s_c) + ((q^4-1)/q^3)*s_bc + (((q^2-1)/q^2)*s_d)*s_bcd + 1/q^2*s_cd*s_b_c_d
s_d_a_b*s_ad
s_bc_ad*s_ad
s_c_d_a*s_cd : ((-q^2+1)*s_a*s_d) + ((-q^4+1)/q)*s_ad + ((-q^2+1)*s_c)*s_acd + q^2*s_cd*s_c_d_a
s_d_a_b*s_ad : ((-q^2+1)*s_a*s_b) + ((-q^4+1)/q)*s_ab + ((-q^2+1)*s_d)*s_abd + q^2*s_ad*s_d_a_b
s_bc_ad*s_ad : s_ad*s_bc_ad
s_c_d_a*s_ad : (((q^2-1)/q^2)*s_c*

s_acd*s_b_cd_a
s_abc*s_b_cd_a : (((q^2-1)/q)*s_b*s_abcd)*s_ad + (((-q^2+1)/q)*s_a*s_abcd)*s_b_c_d + (((-q^2+1)/q)*s_b*s_d)*s_bcd + (((q^2-1)/q)*s_a*s_d)*s_acd + ((q^4-1)/q^2)*s_ad*s_acd + ((-q^4+1)/q^2)*s_b_c_d*s_bcd + s_b_cd_a*s_abc
s_abd*s_b_cd_a
s_ac*s_c_da_b
s_bd*s_a_bc_d : ((q^3-q)*s_c*s_d*s_abcd+((q^2-1)/q^2)*s_a*s_b) + ((q^4-1)/q^3)*s_ab + ((q^3-q)*s_a*s_d)*s_bd + (((q^4-1)/q^2)*s_c)*s_abc + ((q^4-q^2)*s_d)*s_abd + ((-q^4+1)/q^2)*s_bc*s_a_b_c + ((-q^3+q)*s_d)*s_a_b_c*s_bcd + q^2*s_a_bc_d*s_bd
s_bd*s_c_da_b
s_abc*s_c_da_b
s_acd*s_b_cd_a : ((-q^2+1)*s_a*s_abcd) + ((-q^2+1)*s_b)*s_cd + ((-q^4+1)/q)*s_bcd + q^2*s_b_cd_a*s_acd
s_bcd*s_c_da_b
s_abd*s_b_cd_a : (((-q^2+1)/q)*s_a*s_abcd)*s_bc + (((q^2-1)/q)*s_b*s_abcd)*s_c_d_a + (((-q^2+1)/q)*s_b*s_c)*s_bcd + (((q^2-1)/q)*s_a*s_c)*s_acd + ((-q^4+1)/q^2)*s_bc*s_bcd + ((q^4-1)/q^2)*s_c_d_a*s_acd + s_b_cd_a*s_abd
s_ac*s_b_cd_a : (((q^2-1)/q^2)*s_b*s_c+(q^3-q)*s_a*s_d*s_abcd) + ((q^4-1)/q^3)*s_bc + ((q^3-q)*s_a*s_b)*s_ac + ((q^4-q^2)*s_a)*s_

Process ForkProcess-6:
Process ForkProcess-7:
Process ForkProcess-4:
Process ForkProcess-3:
Process ForkProcess-1:
Traceback (most recent call last):
  File "/home/sage/sage/local/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/sage/sage/local/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/sage/sage/local/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/sage/sage/local/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/sage/sage/local/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/sage/sage/local/lib/python3.7/concurrent/futures/process.py", line 226, in _process_worker
    call_item = call_queue.get(blo

  File "/home/sage/sage/local/lib/python3.7/concurrent/futures/process.py", line 226, in _process_worker
    call_item = call_queue.get(block=True)
KeyboardInterrupt
  File "/home/sage/sage/local/lib/python3.7/multiprocessing/queues.py", line 93, in get
    with self._rlock:
  File "sage/rings/fraction_field_element.pyx", line 121, in sage.rings.fraction_field_element.FractionFieldElement.__init__ (build/cythonized/sage/rings/fraction_field_element.c:3073)
    self.reduce()
KeyboardInterrupt
  File "/home/sage/sage/local/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "src/cysignals/signals.pyx", line 320, in cysignals.signals.python_check_interrupt
KeyboardInterrupt
  File "sage/rings/fraction_field_element.pyx", line 1190, in sage.rings.fraction_field_element.FractionFieldElement_1poly_field.reduce (build/cythonized/sage/rings/fraction_field_element.c:12485)
    self.normalize_leading_coefficients()
  File "sage/rings/f

s_a_bc_d*s_b_c_d : (((-q^2+1)/q^3)*s_c*s_d*s_abcd+((q^2-1)/q^2)*s_a*s_b) + ((q^4-1)/q^3+((-q^2+1)/q^3)*s_d^2)*s_ab + (((-q^2+1)/q^4)*s_d)*s_abd + (((q^2-1)/q^3)*s_d)*s_cd*s_abc + 1/q^2*s_b_c_d*s_a_bc_d
s_c_da_b*s_c_d_a : ((q^3-q)*s_c*s_d*s_abcd+(-q^2+1)*s_a*s_b) + ((-q^4+1)/q+(q^3-q)*s_c^2)*s_ab + ((q^4-q^2)*s_c)*s_abc + ((-q^3+q)*s_c)*s_cd*s_abd + q^2*s_c_d_a*s_c_da_b
s_c_da_b*s_ab : (((-q^2+1)/q)*s_b*s_d*s_abcd+((q^2-1)/q^2)*s_a*s_c) + (((-q^2+1)/q)*s_b*s_c)*s_ab + ((q^4-1)/q^3)*s_c_d_a + ((-q^2+1)*s_b)*s_abc + 1/q^2*s_ab*s_c_da_b + (((q^2-1)/q)*s_b)*s_cd*s_abd
s_a_bc_d*s_cd : (((-q^2+1)/q)*s_b*s_d*s_abcd+((q^2-1)/q^2)*s_a*s_c) + (((-q^2+1)/q)*s_a*s_d)*s_cd + ((q^4-1)/q^3)*s_a_b_c + ((-q^2+1)*s_d)*s_acd + (((q^2-1)/q)*s_d)*s_ab*s_bcd + 1/q^2*s_cd*s_a_bc_d
s_c_da_b*s_cd : ((-q^2+1)*s_b*s_d+((q^2-1)/q)*s_a*s_c*s_abcd) + (((q^2-1)/q)*s_b*s_c)*s_cd + ((-q^4+1)/q)*s_d_a_b + (((q^2-1)/q^2)*s_c)*s_bcd + (((-q^2+1)/q)*s_c)*s_ab*s_acd + q^2*s_cd*s_c_da_b
s_c_da_b*s_ab_cd : (((-q^4+2*q^2-1)/q^

KeyboardInterrupt: 

#### Generate  other Higher Relations

Autogenerate relations from the relations which are not commutativity and commutator relations.

In [ ]:
def m_term_to_reduce(expr):
    """Select most important monomial containing an m_term from expr."""
    mmlst = []
    lst = []
    for i in expr.monomials():
        m_count = len(set(terms_of_product(i)).intersection(set(m_terms)))
        if m_count > 0:
            #add to lst if monomial contains an m_term
            lst.append((m_count, i))
            #return monomial with most m_terms and if joint largest monomial
    return (sorted(lst).pop())[1]

In [ ]:
m_term_to_reduce(s_ab*s_ab_cd + s_b_c_d)

In [ ]:
def generate_higher_from_rel(k, relations_system=basic_rw, rel_queue=None):
    """Given a key k=ab generate all relations which are required to resolve the ambiguity (ma)b = m(ab)."""
    print(f"generating relations from {k}")
    rel_set = []
    while rel_queue is not None and not rel_queue.empty():
        rel_set.append(rel_queue.get())
    rel_set = set(rel_set)
    gen_rels = {}
    a,b = terms_of_product(k)
    rel = relations_system[k]
    for m in A.gens():
        if m*a in m_relations:
            print(m*a*b)
            m1 = m_term_to_reduce(m_relations[m*a])
            term = m1*b
            if (not term in gen_rels and not term in rel_set and len(terms_of_product(term))==2):
                print(term)
                l = m_relations[m*a]*b
                r = reduce(m*rel, reduced_commutator_rw)
                new_rel = make_subject(l,r, term)
                print(new_rel)
                gen_rels.update(new_rel)
    return {k: x for k, x in gen_rels.items() if len(terms_of_product(k)) == 2}

In [ ]:
import multiprocessing as mp

def rel_queue(rels):
    """Add relations to queue."""
    q = mp.Manager().Queue()
    for rel in rels:
        q.put(rel)
    return q

def generate_higher_rels(rel_system=basic_m_rels, original_rels={}):
    """Apply generate_higher_from_rel(k) for all relevant k=ab."""
    system = {**original_rels, **rel_system}
    queues = {k: rel_queue(original_rels) for k in rel_system.keys()}
    with ProcessPoolExecutor(8) as pool:
        future_to_key = {pool.submit(generate_higher_from_rel, k, system, queues[k]): k
            for k in rel_system.keys()}
        rels = {**original_rels}
        for higher in futures.as_completed(future_to_key):
            print(f"finished generating relations from {future_to_key[higher]}")
            try:
                rels.update(higher.result())
            except:
                raise 
            del queues[future_to_key[higher]]
            for q in queues.values():
                for k in higher.result():
                    q.put(k)
    return rels

In [ ]:
higher_relations = generate_higher_rels(simple_relations, {})

In [ ]:
len(higher_relations)

In [ ]:
all_relations = {**reduced_commutator_rw.rels, **higher_relations}
all_rw_system = RewritingSystem(all_relations)

In [ ]:
len(all_relations)

In [ ]:
%%time
pool = ProcessPoolExecutor(8)
reduced_rw = all_rw_system.reduce_all()

In [ ]:
#minimal_m_rels = { k:m_relations[k] for k in filter(lambda k: is_ordered(k) and is_s_term_ordered(k), m_relations.keys()) }
#s_bc*s_ab_cd in minimal_m_rels

In [ ]:
extra = generate_higher_rels(m_relations, reduced_rw.rels)

In [ ]:
with_extra = {**extra, **reduced_rw.rels}

In [ ]:
len(with_extra)

In [ ]:
[b*a for a, b in combinations(A.gens(), 2) if not b*a in with_extra]

In [ ]:
[a*b for a, b in combinations(A.gens(), 2) if not a*b in with_extra]

In [ ]:
len([a*b for a, b in combinations(A.gens(), 2) if not a*b in with_extra])

In [ ]:
extra_rw = RewritingSystem(with_extra)

In [ ]:
%%time
pool = ProcessPoolExecutor(8)
reduced_rw = extra_rw.reduce_all()

### Reducable Check

In this section we check that we have generated all the rewriting rules that we expected to generate.

In [ ]:
def loop_group(a):
    """Returns the type of loop."""
    point_lst = loop_point_groups(a)
    if len(point_lst) == 1:
        if len(point_lst[0]) == 3:
            return 'triple'
        if point_lst[0] in ['ac', 'bd']:
            return 'cross'
        else:
            return 'double'
    if len(point_lst) == 2:
            return 'commutative'
    if len(point_lst) == 3:
        if len(point_lst[1]) == 1:
            return '1miss'
        if len(point_lst[1]) == 2:
            return '2miss'
    return

In [ ]:
def loop_group_number(a):
    """Returns group number of loop."""
    group = {'double': 1, '1miss': 2, 'commutative': 2, '2miss': 3, 'cross': 4, 'triple':5}
    return group[loop_group(a)]

In [ ]:
def reducable(mon):
    """Tests if mon should be reducable."""
    #assumes mon = a*b
    a,b = terms_of_product(mon)
    if a > b:
        return True
    if loop_group(a) == loop_group(b):
        return True
    a_terms = terms_of_product(s_term_prod(a))
    b_terms = terms_of_product(s_term_prod(b))
    if set(a_terms).issubset(set(b_terms)):
        return False
    if loop_group(b) == 'triple':
        rest = [s for s in a_terms if not letters(s).issubset(letters(b))]
        if len(rest) == 0:
            return False
        if len(rest) == 1 and rest[0] in [s_ab, s_bc, s_cd, s_ad]:
            return False
    if loop_group(b) == 'cross' and (loop_group(a) == 'double' or loop_group(a) == 'commutative'):
        return False
    if loop_group(b) == 'cross' and len(loop_point_groups(a)) == 3 and set(loop_point_groups(a)[1]).issubset(letters(b)):
        return False
    else:
        return True

Check that all $ab$ which should be reducable have an associated rewriting rule.

In [ ]:
[a*b for a, b in combinations(A.gens(), 2) if not a*b in with_extra and reducable(a*b)]

Check that all reduced $ab$ do not have an associated rewriting rule.

In [ ]:
[a*b for a, b in combinations(A.gens(), 2) if a*b in with_extra and not reducable(a*b)]

# Checking Ambiguities

We now test all the ambiguities for the term rewriting system `reduced_rw`. To make it easier to deal with errors this is done in stages but there is a function `all_tests()` which will run all the tests at once.

In [ ]:
def _amb_check1(t):
    i,j,k = t
    return t, check_amb(i,j,k, reduced_rw, 100)

In [ ]:
def three_puncture_test1():
    """Tests all the ambiguities for the term rewriting system of the subalgebra containing only three points a,b,c."""
    gen = [s_ab, s_bc, s_ac, s_a_b_c, s_abc]
    
    futs = (pool.submit(_amb_check1, arg) for arg in permutations(gen, int(3)))
    for counter, fut in enumerate(futures.as_completed(futs), 1):
        args, res = fut.result()
        print(args)
        print(res)
        print(counter)
    return "end"

In [ ]:
def three_puncture_gen_triples():
    lst_abc = [s_ab, s_bc, s_ac, s_a_b_c, s_abc]
    lst_abd = [s_ab, s_bd, s_ad, s_d_a_b, s_abd]
    lst_acd = [s_ac, s_cd, s_ad, s_c_d_a, s_acd]
    lst_bcd = [s_bc, s_cd, s_bd, s_b_c_d, s_bcd]
    return flatten([list(permutations(gen, int(3))) for  gen in [lst_abc, lst_abd, lst_acd, lst_bcd]], list)

def all_three_puncture_tests():
    """Tests all the ambiguities for each of the four subalgebras containing only three points."""
    choices = three_puncture_gen_triples()
    
    futs = (pool.submit(_amb_check1, arg) for arg in choices)
    for counter, fut in enumerate(futures.as_completed(futs), 1):
        args, res = fut.result()
        print(args)
        print(res)
        print(counter)
    return "end"

In [ ]:
%%time
#480 tests in total
pool = ProcessPoolExecutor(8)
all_three_puncture_tests()

In [ ]:
#This is unnecessary as all these tests are convered in the next test.
def doubles_and_triples_tests():
    """Tests all ambiguities only involving simple generators."""
    choices = permutations(simple_terms, int(3))
    
    futs = {pool.submit(_amb_check1, arg): arg for arg in choices}
    failed_tests = []
    for counter, fut in enumerate(futures.as_completed(futs.keys()), 1):
        try:
            args, res = fut.result()
        except:
            print(f"error for arg {futs[fut]}")
        print(args)
        print(res)
        print(counter)
        if not isinstance(res, str):
            failed_tests.append(args)
        if len(failed_tests)>50:
            print(failed_tests)
            return "max failed tests exceeded"
    return "end"

In [ ]:
#%%time
#pool = ProcessPoolExecutor(8)
#doubles_and_triples_tests()

In [ ]:
non_central = [s for s in A.gens() if 1<len(letters(s))<4]

def all_other_tests():
    """Test all ambiguities which are not covered by all_three_puncture_tests()."""
    already_counted = three_puncture_gen_triples()
    choices = filter(lambda x: not x in already_counted, permutations(non_central, int(3)))
    
    futs = (pool.submit(_amb_check1, arg) for arg in choices)
    failed_tests = []
    for counter, fut in enumerate(futures.as_completed(futs), 1):
        try:
            args, res = fut.result()
        except:
            print(f"error for arg {futs[fut]}")
        print(args)
        print(res)
        print(counter)
        if not isinstance(res, str):
            failed_tests.append(args)
        if len(failed_tests)>50:
            print(failed_tests)
            return "max failed tests exceeded"
    return "end"

In [ ]:
%%time
pool = ProcessPoolExecutor(8)
all_other_tests()

In [ ]:
def run_failed_tests():
    futs = (pool.submit(_amb_check1, arg) for arg in failed_test_pool)
    failed = 0
    passed = 0
    for counter, fut in enumerate(futures.as_completed(futs), 1):
        args, res = fut.result()
        print(args)
        print(res)
        print(counter)
        if not isinstance(res, str):
            failed = failed + 1
        else:
            passed = passed + 1
    print(f"Failed {failed} test and passed {passed} test out of a total of {len(failed_test_pool)} tests")
    return

In [ ]:
#%%time
#pool = ProcessPoolExecutor(8)
#run_failed_tests()

In [ ]:
def all_tests():
    """Test all ambiguities in one go."""
    choices = permutations(non_central, int(3))
    
    futs = (pool.submit(_amb_check1, arg) for arg in choices)
    failed_tests = []
    for counter, fut in enumerate(futures.as_completed(futs), 1):
        try:
            args, res = fut.result()
        except:
            print(f"error for arg {futs[fut]}")
        print(args)
        print(res)
        print(counter)
        if not isinstance(res, str):
            failed_tests.append(args)
        if len(failed_tests)>50:
            print(failed_tests)
            return "max failed tests exceeded"
    return "end"

In [ ]:
#%%time
#pool = ProcessPoolExecutor(8)
#all_tests()

# Other Checking Code

Below is all the code which does not relate to checking the the term rewriting system is unambiguous but other things which need to be checked e.g. termination.

### Partial Ordering
We construct a partial order and check that the partial order is compatible with the term rewriting system we have constructed. This partial order is `lt_combined` which is constructed by chaining three partial orders `le_inversion`, `le_total` and `le_nearness`.

In [ ]:
def degree(expr):
    """Find the degree of expr assuming each variable is of degree 1."""
    return max([ len(terms_of_product(expr)) for expr in expr.monomials()])

def inversion_number(mon):
    """Count the number of pairwise terms in the monomial mon which are in the incorrect order."""
    terms = terms_of_product(to_gen_str(mon))
    return sum([1 for i,j in combinations([0..len(terms)-1], 2) if terms[i] > terms[j]])

def degree_n_inversion(expr, n):
    """Count the number of inversions of all monomials in expr of length n."""
    return sum([inversion_number(w) for w in expr.monomials() if len(terms_of_product(w)) == n ])

def safe_max(ls):
    """Safe maximum function which returns 0 if the list in empty."""
    if len(ls):
        return max(ls)
    return 0

def max_nonzero(ls):
    """Return index of last nonzero element of lst."""
    return safe_max([i for i, x in enumerate(ls) if x != 0])

In [ ]:
def le_inversion(expr1, expr2):
    """Returns if expr1 is less than or equal to expr2 according to the reduced degree ordering."""
    inv_degree_1 = max_nonzero([degree_n_inversion(expr1, n) for n in [0..degree(expr1)]])
    inv_degree_2 = max_nonzero([degree_n_inversion(expr2, n) for n in [0..degree(expr2)]])
    if inv_degree_1 < inv_degree_2:
        return True
    if (inv_degree_1 == inv_degree_2 
        and degree_n_inversion(expr1, inv_degree_1) <= degree_n_inversion(expr2, inv_degree_2)):
        return True
    return False

In [ ]:
def lt_inversion(expr1, expr2):
    """Returns if expr1 is strictly less than expr2 according to the reduced degree ordering."""
    if le_inversion(expr1, expr2):
        return not [degree_n_inversion(expr1, n) for n in [0..degree(expr1)]] == [degree_n_inversion(expr2, n) for n in [0..degree(expr2)]]
    return False

In [ ]:
def generator_degree(s):
    """Finds the degree of generator s which is used in the grading."""
    if s == 1:
        return 0
    degrees = {'double': 2, 'cross': 2, 'triple': 3, '1miss': 4, 'commutative': 4, '2miss': 6}
    return degrees[loop_group(s)]

def monomial_degree(m):
    """Finds the graded degree of the monomial mon."""
    return sum([generator_degree(m) for m in terms_of_product(to_gen_str(m))])

def total_degree(expr):
    """Finds the graded degree of expr."""
    return max([monomial_degree(m) for m in expr.monomials()])

def highest_degree(expr):
    """Return a list of monomials of maximal degree."""
    N = total_degree(expr)
    return [m for m in expr.monomials() if total_degree(m) == N]

In [ ]:
def lt_total(expr1, expr2):
    """Returns if expr1 is strictly less than expr2 according to the total degree ordering."""
    if total_degree(expr1) < total_degree(expr2):
        return True
    if (total_degree(expr1) == total_degree(expr2) 
        and all([m1 > m2 for m1 in expr1.monomials() for m2 in expr2.monomials()])):
        return True
    return False

def le_total(expr1, expr2):
    """Returns if expr1 is less than or equal to expr2 according to the total degree ordering."""
    return True if lt_total(expr1, expr2) or total_degree(expr1)==total_degree(expr2) else False

In [ ]:
def number_distinct(expr):
    """Finds the maximal number of distint loops in the maximal degree terms of expr."""
    n = total_degree(expr)
    return max([len(terms_of_product(x)) for x in expr.monomials() if total_degree(x) == n])

def group_distance(x, y):
    """Finds the distance between loops x and y where distance is defined depending on the group of the loop."""
    return abs(loop_group_number(loop_group(x)) - loop_group_number(loop_group(y)))

def nearness_mon(mon):
    """Returns a measure of the pairwise nearness of the loops in the monomial mon."""
    return sum([group_distance(x, y) for x,y in combinations(terms_of_product(mon),2)])

def nearness(expr):
    """Returns a measure of the pairwise nearness of the loops in expr."""
    deg = total_degree(expr)
    return sum([nearness_mon(m) for m in expr.monomials() if total_degree(m) == deg])

In [ ]:
def lt_nearness(expr1, expr2):
    """Returns if expr1 is strictly less than expr2 according to the group distance measure."""
    deg1, deg2 = (total_degree(expr1), total_degree(expr2))
    if deg1 != deg2:
        return deg1 < deg2
    if number_distinct(expr1) != number_distinct(expr2):
        return number_distinct(expr1) < number_distinct(expr2)
    print(nearness(expr1), nearness(expr2))
    return nearness(expr1) > nearness(expr2)

In [ ]:
def lt_combined(expr1, expr2):
    """Returns if expr1 is strictly less than expr2 according to the combined partial order."""
    if (lt_inversion(expr1, expr2) 
        or (le_inversion(expr1, expr2) and lt_total(expr1, expr2)) 
        or (le_inversion(expr1, expr2) and le_total(expr1, expr2) and lt_nearness(expr1, expr2))):
        return True
    else:
        return False

Test term rewriting system is compatible with partial order

In [ ]:
def test_order_compatibility(rws=reduced_rw, order=lt_combined):
    """Test if the rewriting system rws is compatible with the partial ordering order."""
    return len([k for k in rws if not order(rws[k], k)]) == 0

Code Tests

In [ ]:
degree(s_a*s_b + s_ab*s_bc + s_abc) == 2

In [ ]:
inversion_number(s_ab*s_abc) == 0

In [ ]:
inversion_number(s_abc*s_ab) == 1

In [ ]:
inversion_number(s_bc^2*s_bd) == 0

In [ ]:
max_nonzero([]) == 0

In [ ]:
max_nonzero([0,1,1]) == 2

In [ ]:
max_nonzero([0, 0]) == 0

In [ ]:
le_inversion(s_bc*s_ab, s_cd) == False

In [ ]:
le_inversion(s_ab*s_bc, s_cd) == True

In [ ]:
le_inversion(s_a_b_c*s_ab, s_ab*s_a_b_c) == False

In [ ]:
monomial_degree(s_ab*s_abc) == 5

In [ ]:
total_degree(s_ab*s_abc + s_bc*s_a_bc_d) == 8

### Form of Reduced Monomials
In this section we check that the reduced monomials are of the expected type. 

In [ ]:
def is_reduced(expr):
    """Test if expr is reduced."""
    return reduced_rw.replace(expr) == expr

In [ ]:
def all_group_number(n):
    """Return a list of all loops of group number n."""
    return [x for x in A.gens() if loop_group_number(x) == n]

def is_subsetting(a, b):
    """Test if the loops in the monomial a are a subset of the loops in b."""
    return set(terms_of_product(s_term_prod(a))).issubset(set(terms_of_product(s_term_prod(b))))

In [ ]:
def last_point(s):
    """Return the last letter which is a point of the loop s."""
    return loop_point_groups(s)[-1][-1]

def first_point(s):
    """Return the first letter which is a point of the loop s."""
    return loop_point_groups(s)[0][0]

def missing_points(s):
    """Return all of the points not in the loop s (excluding bar points)."""
    return "".join({'a','b','c','d'} - letters(s))

def is_consecutive(n,m):
    """Test if the points n and m are consecutive points if there are four points in a circle."""
    alph = 'abcd'
    return (alph.index(n) - alph.index(m)) % 4 == 1 or (alph.index(m) - alph.index(n)) % 4 == 1

def end_points(s):
    """Return the points of loop s which start and end it."""
    m = missing_points(s)
    if len(m) == 0:
        #When the loop contains all points the end points are just the first and last point
        return [first_point(s), last_point(s)]
    else:
        #Otherwise the end points are the points next to the missing points
        return [l for l in letters(s) if any([is_consecutive(l,n) for n in m])]

In [ ]:
def standard_loop(s1, m,  s2):
    """Construct the loop which starts with point s1 goes around the points m and ends with the point s2."""
    #Note that s1 and s2 are assumed to be single points as this is the form of the generators
    if m == 'ad':
        m = 'da'
    else:
        m = ''.join(sorted(m))
    try: 
        loop = eval(f's_{s1}_{m}_{s2}')
    except:
        loop = eval(f's_{s2}_{m}_{s1}')
    return loop

def looping_terms(s):
    """Returns the list of the loop(s) which form a complete loop when multiplied by s."""
    m = missing_points(s)
    #note s 
    s1, s2 = end_points(s)
    if len(m) == 0:
        fl = "".join(sorted(s1+s2))
        return [eval(f"s_{fl}")]
    if loop_group(s) == 'cross':
        return [standard_loop(s1, n, s2) for n in m]
    return [standard_loop(s1, m, s2)]

def complete_loop(a,b):
    """Tests if a*b forms a loop."""
    return set(terms_of_product(s_term_prod(looping_terms(b)[0]))).issubset(set(terms_of_product(s_term_prod(a))))

In [ ]:
#List of all reduced pairs
all_reduced_pairs = [(x,y) for x,y in permutations(A.gens(),int(2)) if is_reduced(x*y)]

#the generating loops split into groups
groups = [all_group_number(n) for n in [1..5]]

In [ ]:
#Test have all reodering relations
len([x*y for x,y in all_reduced_pairs if x>y]) == 0

In [ ]:
#test have relations between all pairs of loops of the same group
all([not is_reduced(a*b) for x in groups for a,b in combinations( x, 2)])

In [ ]:
#test have relaions between all group III and group IV loops
all([not is_reduced(a*b) for a in groups[2] for b in groups[3] ])

In [ ]:
#test if group I, II and III are subsetting then they are reduced
all([is_reduced(a*b)
     for i,j in [(0, 1), (0,2), (1,2)] for a in groups[i] for b in groups[j] if is_subsetting(a,b)])

In [ ]:
#test if group I, II and III are not subsetting then they are not reduced
all([not is_reduced(a*b)
     for i,j in [(0, 1), (0,2), (1,2)] for a in groups[i] for b in groups[j] if not is_subsetting(a,b)])

In [ ]:
# all groups I,II, III loops which do not complete the loop are reduced
all([is_reduced(a*b) for a in flatten([groups[0], groups[1], groups[2]]) for b in groups[4] if not complete_loop(a,b) ])

In [ ]:
# all groups I,II, III loops which complete the loop are not reduced
all([not is_reduced(a*b) for a in flatten([groups[0], groups[1], groups[2]]) for b in groups[4] if complete_loop(a,b) ])

In [ ]:
# if the type IV loop is a subset of the type V loop then it is reduced
all([is_reduced(a*b) for a in groups[3] for b in groups[4] if letters(a).issubset(letters(b))])

In [ ]:
# if the type IV loop is not a subset of the type V loop then it is not reduced
all([not is_reduced(a*b) for a in groups[3] for b in groups[4] if not letters(a).issubset(letters(b))])

Code Tests

In [ ]:
last_point(s_d_a_b) == 'b'

In [ ]:
last_point(s_acd) == 'd'

In [ ]:
first_point(s_d_a_b) == 'd'

In [ ]:
missing_points(s_ab) == 'dc'

In [ ]:
is_consecutive('d','b') == False

In [ ]:
is_consecutive('a','d') == True

In [ ]:
end_points(s_d_ab_c)

In [ ]:
end_points(s_abc)

In [ ]:
standard_loop('b', 'dc', 'a') == s_b_cd_a

In [ ]:
standard_loop('a', 'dc', 'b') == s_b_cd_a

In [ ]:
looping_terms(s_a_bc_d) == s_ad

In [ ]:
looping_terms(s_a_b_c) == s_cd*s_ad

## Hilbert Series

We now check that the Hilbert series of the Skein algebra `hilbert_series_skein` when n=4 agrees with the Hilbert series `new_hilbert` of the algebra whose presentation we have constructed.

In [ ]:
var('t')
var('m')
var('n')

In [ ]:
 the new Hilbert sdef binomial_replace(n, m):
    """Number of choices of m from n with replacement."""
    return binomial(n+m-1, m)

def hilbert_series_skein(n):
    """The Hilbert series of the skein algebra for n+1 punctures"""
    return (1+t)^n/(1-t)^n*(hypergeometric([n,n],[1], t^2) - n*t*hypergeometric([n,n+1],[2], t^2))

In [ ]:
#Parts of the new Hilbert series
central_hilb = 1/((1-t^4)*(1-t)^4)
III_term = (1-t^8)/(1-t^2)^4
III_V_term = t^3/(1-t^3)*(1-t^4)/(1-t^2)^4
IV_term = t^2/(1-t^2)*( 1 + 2*t^3/(1-t^3) )*( 1 + 4*t^2/(1-t^2) + 4*t^4/(1-t^2)^2)

#Combined together
new_hilbert = central_hilb*(III_term + 4*III_V_term + 2*IV_term)

In [ ]:
# Test the Hilbert series match
((hilbert_series_skein(4).simplify_hypergeometric()) - new_hilbert).full_simplify()

### Relations Reduce

Finally, we check that the we can eliminate the extra generators and when we do so we only have the expected relations.

The free algebra used so far is A which is an algebra over P. We define a new free algebra B with only the reduced generators which is an algebra over R.

In [ ]:
R.<r_a,r_b,r_c, r_d, r_abcd> = PolynomialRing(F,5,order='neglex')

In [ ]:
variablesB = ('r_ab, r_bc, r_cd, r_ad, '
             'r_ac, r_bd, r_abc, r_bcd, r_acd, r_abd').split(', ')
B = FreeAlgebra(R, order='neglex', names=variablesB)
for k, v in zip(variablesB, B._first_ngens(len(variablesB))):
    locals()[k] = v

In [ ]:
def reverse_m_rels():
    """Reverses the m_rels to give relations which can eliminate the extra generators."""
    rels = [make_subject(k, r, m_term_to_reduce(r)) for  k,r in m_relations.items() if is_ordered(k)]
    rels + [{s_ab_cd: s_ab*s_cd}, {s_bc_ad: s_bc*s_ad}]
    return functools.reduce((lambda x, y: {**x, **y}), rels, {})

In [ ]:
reverse = RewritingSystem(reverse_m_rels())

In [ ]:
%%time
pool = ProcessPoolExecutor(8)
reduced_reverse = reverse.reduce_all_seq()

In [ ]:
ordered_reduced_reverse = RewritingSystem({k: reduce(reduced_reverse[k], RewritingSystem(reordering)) 
                                           for k in reduced_reverse})

In [ ]:
def basis_mapping(expr):
    """Map the extended generators of A to expressions in B."""
    #eliminate extra generators using the rules in ordered_reduced_reverse
    expr = reduce(expr, ordered_reduced_reverse)
    #map the terms to elements of B
    return eval(re.sub('s','r', str(to_gen_str(expr))))

def relation_mapping(rels):
    """Convert a reduction rule for A to a reduction rule for B."""
    m_rel = {}
    #convert the expressions in rels to elements of B
    mapped = {basis_mapping(k): basis_mapping(rels[k]) for k in rels}
    #for each relation rearrange the relation so the key is a single monomial again
    for k in mapped: 
        d = max( [len(terms_of_product(x)) for x in k.monomials()])
        term = [x for x in k.monomials() if len(terms_of_product(x)) == d][0]
        print(make_subject(k, mapped[k], term))
        m_rel.update(make_subject(k, mapped[k], term))
    return m_rel

In [ ]:
#The two cases are to prevent overcounting of triple or quadratic relations but reordering the key 
basic_rels = {k:r for k,r in relation_mapping(simple_relations).items() if (len(terms_of_product(k)) == 2 or is_ordered(k)) }

basic_commutators = relation_mapping({a*b: reduced_rw[a*b] for a,b in permutations(simple_terms, int(2)) 
                                      if a*b in reordering})

#TRWS containing all the relations which pertain to the reduced basis
base_rw = RewritingSystem({**basic_rels, **basic_commutators})

In [ ]:
#Check correct number of relations
len(base_rw) == 65

In [ ]:
%%time
pool = ProcessPoolExecutor(8)
reduced_base = base_rw.reduce_all()#Construct a list of all the relations for A mapped to relations of B

Construct a list of all the relations for A mapped to relations of B

In [ ]:
algebra_relations = []
for k in basic_m_rw:
    print(basis_mapping(k) - basis_mapping(basic_m_rw[k]))
    algebra_relations = algebra_relations + [basis_mapping(k) - basis_mapping(basic_m_rw[k])]

Test that all relations of A are satisfied in B

In [ ]:
for eq in algebra_relations:
    to_solve = []
    red = reduce(eq, reduced_base)
    print(print(red))
    if red != 0:
        to_solve = to_solve + [red]

We must show that eq is 0 in B. 
The term $r_{ab} r_{bc} r_{cd} r_{ac}$ can be simplified by the relation $r_{ab} r_{bc} r_{cd} r_{ac}$ but the $s_{cd}$ term stops the TRWS automatically reducing it.

In [ ]:
eq = ((((q^8-q^6+q^2-1)/q^4)*r_c*r_d+(-q^4+1)*r_b^2*r_c*r_d+((q^8-q^6+q^2-1)/q^3)*r_a*r_b*r_abcd
       +(q^4-q^2)*r_a^2*r_c*r_d) + ((q^6+q^4-q^2-1)*r_abcd+(q^5-q^3)*r_a*r_b*r_c*r_d)*r_ab 
      + (((-q^4+1)/q)*r_b*r_d)*r_bc + ((-2*q^8-2*q^6+q^4-1)/q^5+q^3*r_c^2+q*r_b^2+q^5*r_a^2)*r_cd 
      + (((q^2-1)/q^3)*r_a*r_c)*r_ad + ((q^4-q^2)*r_b*r_c*r_abcd+(q^5-q)*r_a*r_d)*r_ac 
      + (((-q^6+1)/q)*r_b*r_c)*r_bd + ((q^3-q)*r_c*r_abcd+(q^4-q^2)*r_a*r_b*r_d)*r_abc 
      + (((-q^6+q^4+q^2-1)/q^4)*r_b+(-q^4+q^2)*r_b*r_c^2)*r_bcd + (((q^10-q^6+q^4-1)/q^4)*r_a)*r_acd 
      + ((-q^2+1)*r_a*r_b*r_c)*r_abd + ((q^6-q^4)*r_c*r_d)*r_ab^2 + q^6*r_a*r_b*r_ab*r_cd 
      + ((q^4-1)*r_b*r_c)*r_ab*r_ad + ((q^4-q^2)*r_b*r_d)*r_ab*r_ac + ((q^5-q)*r_d)*r_ab*r_abc 
      + ((q^5-q)*r_b)*r_ab*r_acd + ((q^5-q^3)*r_c)*r_ab*r_abd + q^4*r_b*r_c*r_bc*r_cd + ((-q^8+1)/q^4)*r_bc*r_bd 
      + ((-q^3+q)*r_c)*r_bc*r_bcd + (((-q^4+1)/q^3)*r_a)*r_bc*r_abd + q^4*r_a*r_c*r_cd*r_ac 
      + q^3*r_a*r_b*r_c*r_cd*r_abc + ((q^5-q^3)*r_c)*r_ac*r_acd + q^7*r_ab^2*r_cd 
      + ((q^4-1)/q)*r_ab*r_bc*r_ad + q^4*r_c*r_ab*r_cd*r_abc + q^3*r_bc^2*r_cd + q^2*r_a*r_bc*r_cd*r_abc 
      + q^3*r_cd*r_ac^2 + q^2*r_b*r_cd*r_ac*r_abc + q*r_cd*r_abc^2 + (-q^4)*r_ab*r_bc*r_cd*r_ac)

Construct relation for $r_{ab}r_{bc}r_{cd}r_{ac}$

In [ ]:
l = r_cd*reduce(r_ab*r_bc*r_ac, reduced_base)
r = reduce(reduce(r_cd*r_ab, reduced_base)*r_bc*r_ac, reduced_base)
extra_rel = make_subject(l, r, r_ab*r_bc*r_cd*r_ac)

Now it reduces

In [ ]:
reduce(reduce(eq, RewritingSystem(extra_rel)), reduced_base)